In [1]:
import json
import pandas as pd
import numpy as np

In [24]:
parameters = {}
with open("parameters.json") as file:
    parameters = json.load(file)
parameters

{'440': [8.28691072930097,
  16.969144954176567,
  -0.03762955826175123,
  0.7144584646211575,
  438.83268710508725],
 '470': [7.48663394846013,
  22.39832769279835,
  -0.026126347658920756,
  0.5820076799259942,
  469.1396127982954],
 '495': [6.3097280786613785,
  24.622045097754327,
  -0.031336153632372556,
  0.4685330184110434,
  493.8408066569592],
 '530': [5.375242537758548,
  30.926993468698544,
  -0.02239693371436714,
  0.47141314468491813,
  522.5961992949494],
 '595': [2.141166878502169,
  14.954282181134916,
  0.025261762229087867,
  0.6132208833495716,
  596.5676235390304],
 '634': [5.28163593978372,
  16.733745798113073,
  0.04741898411571205,
  0.5594236100268427,
  635.223104551172],
 '660': [5.145963326544925,
  19.062221737422384,
  0.04776843377911945,
  0.5570733930427237,
  660.2844490143182],
 '684': [8.348609961964955,
  23.801113955671234,
  0.03708604960960036,
  0.40126681820255994,
  683.3105800770115],
 'branco': [2.012455723944545,
  11.41406112175179,
  20.6

In [4]:
neutral_spectrum_components = {}
with open("neutral_spectrum_components.json") as file:
    neutral_spectrum_components = json.load(file)["coefficients"]

In [5]:
neutral_spectrum_components

{'440': 0.2345996831531853,
 '470': 0.3009961550222661,
 '495': 0.37546771741691637,
 '530': 0.28211687274917013,
 '595': 0.17609814440797206,
 '634': 0.1490444072029084,
 '660': 0.2145333049075533,
 '684': 0.30246412667317074,
 'branco': 0.9999537477523128,
 'full': 0.23291515239948163}

In [6]:
validation = pd.read_csv("validation/validation.csv",sep="\t")

In [9]:
validation = validation.set_index("painel")

In [10]:
validation

,1,2,3,r1,r2
painel,,,,,
branco,13,9.12,9.17,12.85,13.7
full,13,9.42,9.40,12.50,12.8
440,8,9.02,9.06,7.82,7.6
470,7,8.55,8.57,6.78,6.2
495,6,8.52,8.55,5.54,5.0
530,5,8.17,8.28,4.10,4.3
595,1,6.54,6.02,0.20,0.2
634,5,6.19,6.33,4.41,5.0
660,5,6.03,6.07,4.35,4.4


In [11]:
original = {}
with open("irradiancia2.json") as jfile:
    original = json.load(jfile)

In [12]:
original

{'440': 8,
 '470': 7,
 '495': 6,
 '530': 5,
 '595': 2,
 '634': 5,
 '660': 5,
 '684': 8,
 'branco': 13,
 'full': 13}

In [21]:
validation["original"] = pd.Series(original)
validation["components"] = pd.Series(neutral_spectrum_components)

In [22]:
validation

,1,2,3,r1,r2,original,components
painel,,,,,,,
branco,13,9.12,9.17,12.85,13.7,13,0.999954
full,13,9.42,9.40,12.50,12.8,13,0.232915
440,8,9.02,9.06,7.82,7.6,8,0.234600
470,7,8.55,8.57,6.78,6.2,7,0.300996
495,6,8.52,8.55,5.54,5.0,6,0.375468
530,5,8.17,8.28,4.10,4.3,5,0.282117
595,1,6.54,6.02,0.20,0.2,2,0.176098
634,5,6.19,6.33,4.41,5.0,5,0.149044
660,5,6.03,6.07,4.35,4.4,5,0.214533


In [13]:
def G(x,A,b,c,f,x0): return f*((2*A/(np.pi*2*b/(1+np.exp(c*(x-x0)))))/(1+4*((x-x0)/(2*b/(1+np.exp(c*(x-x0)))))**2))+(1-f)*(A/(2*b/(1+np.exp(c*(x-x0)))))*(4*np.log(2)/np.pi)*np.exp(-4*np.log(2)*((x-x0)/(2*b/(1+np.exp(c*(x-x0)))))**2)

In [14]:
def H(x,A0,A1,b0,b1,c0,c1,f0,f1,x00,x01): return G(x,A0,b0,c0,f0,x00) + G(x,A1,b1,c1,f1,x01)

In [15]:
sorted_parameters = sorted(original.items(),key=lambda x: x[0]) #Choosing ordering to make sure the parameters don't get scrambled later

In [25]:
def LinearCombination(x,df,alpha,parameters):
    """
    alpha: Array of values for the coefficients of the linear combination. Must be within the interval [0,1]
    """
    #assert max(alpha)<=1 and min(alpha)>=0
    s = 0
    #for d,a in zip(parameters,alpha):
    for i,row in df.iterrows():
        painel = i
        a = row[alpha]
        d = parameters[painel]
        if painel=="branco" or painel=="full":
            s+=a*H(x,*d)
        else:
            s+=a*G(x,*d)
    return s

    

In [26]:
x = np.linspace(0,1000,100)

In [29]:
original = LinearCombination(x,validation,"components",parameters)
original = LinearCombination(x,validation,"components",parameters)
original = LinearCombination(x,validation,"components",parameters)


array([7.45757102e-05, 7.73188977e-05, 8.02189799e-05, 8.32886589e-05,
       8.65420191e-05, 8.99947204e-05, 9.36642258e-05, 9.75700688e-05,
       1.01734171e-04, 1.06181222e-04, 1.10939127e-04, 1.16039559e-04,
       1.21518616e-04, 1.27417629e-04, 1.33784158e-04, 1.40673219e-04,
       1.48148825e-04, 1.56285927e-04, 1.65172888e-04, 1.74914674e-04,
       1.85637003e-04, 1.97491838e-04, 2.10664729e-04, 2.25384789e-04,
       2.41938484e-04, 2.60689006e-04, 2.82104033e-04, 3.06796335e-04,
       3.35584494e-04, 3.69585939e-04, 4.10363250e-04, 4.60160887e-04,
       5.22300338e-04, 6.01862833e-04, 7.06915405e-04, 8.50812695e-04,
       1.05675121e-03, 1.36738008e-03, 1.86682902e-03, 2.73711097e-03,
       4.42711536e-03, 8.30650658e-03, 2.12556000e-02, 9.33263477e-02,
       1.27671861e-01, 1.15529536e-01, 1.08019059e-01, 8.71552853e-02,
       9.56011994e-02, 1.08872679e-01, 9.18113586e-02, 9.98581734e-02,
       1.05053382e-01, 9.98922600e-02, 9.75346937e-02, 9.75842104e-02,
      